In [1]:
import requests
import bs4
import os
import re
import unicodedata
import pandas as pd
import nltk
import numpy as np

import selenium

In [2]:
url = ('https://www.newegg.com/p/pl?N=100007709%20601361654%20601359415')
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html)

In [3]:
response

<Response [200]>

In [4]:
soup

<!DOCTYPE html>
<html lang="en-us"><head><title>GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards | Newegg.com</title><meta charset="utf-8"/><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="always" name="referrer"/><meta content="GeForce RTX 3060 Desktop Graphics Cards,GeForce RTX 3060 Ti Desktop Graphics Cards" name="keywords"/><meta content="Shop GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards on Newegg.com. Watch for amazing deals and get great pricing." name="description"/><meta content="https://c1.neweggimages.com/WebResource/Themes/Nest/logos/logo_newegg_400400.png" property="og:image"/><meta content="Shop GeForce RTX 3060,GeForce RTX 3060 Ti Desktop Graphics Cards on Newegg.com. Watch for amazing deals and get great pricing." property="og:description"/><meta content="https://www.newegg.com/p/pl?N=100007709%20601359415" property="og:url"/><meta content="website" property="og:type"/><meta content="GeForce RTX 3060,GeForce RTX

In [5]:
items = soup.find_all('div', {'class': 'item-cell'})
items

[<div class="item-cell" id="item_cell_9SIB11NFN06580_1_0"><div class="item-container"><a class="item-img" href="https://www.newegg.com/gigabyte-geforce-rtx-3060-ti-gv-n306teagle-oc-8gd/p/N82E16814932455"><img alt="GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TEAGLE OC-8GD (rev. 2.0)" src="https://c1.neweggimages.com/ProductImageCompressAll300/14-932-455-V01.jpg" title="GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TEAGLE OC-8GD (rev. 2.0)"/></a><div class="item-info"><div class="item-branding"><a class="item-brand" href="https://www.newegg.com/GIGABYTE/BrandStore/ID-1314"><img alt="GIGABYTE" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1314.gif" title="GIGABYTE"/></a></div><a class="item-title" href="https://www.newegg.com/gigabyte-geforce-rtx-3060-ti-gv-n306teagle-oc-8gd/p/N82E16814932455" title="View Details">GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TEAGLE OC

In [6]:
soup.find('a', {'class':'item-title'}).text

'GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 PCI Express 4.0 ATX Video Card GV-N306TEAGLE OC-8GD (rev. 2.0)'

In [7]:
soup.find('li', {'class': 'price-was'}).text

''

In [8]:
soup.find('li', {'class': 'price-current'}).text

'$941.99\xa0(3 Offers)–'

In [55]:
soup.find('p', {'class': 'item-promo'}).text

'Limited time offer, ends 09/30'

In [59]:
items = soup.find_all('div', {'class': 'item-cell'})
type(items[0].find('p', {'class': 'item-promo'}))

NoneType

In [9]:
regex = r'\/(\d+)'
subject = soup.find('span', {'class': 'list-tool-pagination-text'}).text

num_pages = int(re.findall(regex, subject)[0])

In [10]:
num_pages

2

In [11]:
list(range(1,num_pages+1))

[1, 2]

In [12]:
urls = [('https://www.newegg.com/p/pl?N=100007709%20601361654%20601359415&page=' + str(x)) for x in list(range(1,num_pages+1))]

In [19]:
urls

['https://www.newegg.com/p/pl?N=100007709%20601361654%20601359415&page=1',
 'https://www.newegg.com/p/pl?N=100007709%20601361654%20601359415&page=2']

In [79]:
def get_newegg_gpu(urls):
    
    rows = []

    for url in urls:

        response = requests.get(url)
        html = response.text
        soup = bs4.BeautifulSoup(html)

        items = soup.find_all('div', {'class': 'item-cell'})

        for item in items:

            item_title = item.find('a', {'class':'item-title'}).text

            old_price = item.find('li', {'class': 'price-was'}).text

            if old_price == '':
                old_price = '0'
            else:
                old_price = old_price

            current_price = item.find('li', {'class': 'price-current'}).text

            if current_price == '':
                current_price = '0'
            else:
                current_price = current_price
            
            in_stock = item.find('p', {'class': 'item-promo'})
            
            if in_stock is None:
                in_stock = 'available'
            elif in_stock.text == 'OUT OF STOCK':
                in_stock = 'out of stock'
            else:
                in_stock = "sale?"

            row = []
            row.append(item_title)
            row.append(old_price)
            row.append(current_price)
            row.append(in_stock)
            rows.append(row)

        df = pd.DataFrame(columns=['item_title', 'old_price', 'current_price', 'in_stock'], data = rows)
        
    return df

In [80]:
df = get_newegg_gpu(urls)
df

,item_title,old_price,current_price,in_stock
0,GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 P...,0,$941.99 (3 Offers)–,available
1,MSI Gaming GeForce RTX 3060 Ti 8GB GDDR6 PCI E...,"$1,132.82","$1,071.91 (2 Offers)–",available
2,GIGABYTE Gaming GeForce RTX 3060 12GB GDDR6 PC...,$973.82,$956.87 (3 Offers)–,available
3,GIGABYTE AORUS GeForce RTX 3060 12GB GDDR6 PCI...,$969.17,$967.60 (2 Offers)–,available
4,ASUS ROG Strix GeForce RTX 3060 12GB GDDR6 PCI...,0,"$1,144.17 (2 Offers)–",available
...,...,...,...,...
56,GIGABYTE AORUS GeForce RTX 3060 Ti ELITE 8G Gr...,0,$639.99 –,out of stock
57,GIGABYTE GeForce RTX 3060 Ti GAMING PRO 8G Gra...,0,$619.99 –,out of stock
58,GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 P...,0,$449.99 (2 Offers)–,out of stock
59,GIGABYTE GeForce RTX 3060 Ti GAMING PRO 8GB GD...,0,$619.99 –,out of stock


In [81]:
def clean_gpu_data(df):
    df.old_price = df.old_price.str.replace('$', '').str.replace(',', '')
    df['current_price'] = df.current_price.str.split(pat='\xa0', expand=True)
    df.current_price = df.current_price.str.replace('$', '').str.replace(',', '')
    df.current_price = df.current_price.astype(float, errors = 'ignore')
    return df

In [82]:
df = clean_gpu_data(get_newegg_gpu(urls))

In [86]:
df[df.in_stock == 'available'].sort_values('current_price')

,item_title,old_price,current_price,in_stock
15,MAXSUN GeForce RTX 3060 12G GDDR6 Vidoe Gaming...,0,829.99,available
34,"EVGA GeForce RTX 3060 Ti XC GAMING Video Card,...",889.99,859.99,available
0,GIGABYTE Eagle GeForce RTX 3060 Ti 8GB GDDR6 P...,0,941.99,available
3,GIGABYTE AORUS GeForce RTX 3060 12GB GDDR6 PCI...,969.17,967.70,available
31,GIGABYTE GeForce RTX 3060 EAGLE OC 12G Graphic...,0,989.98,available
24,ASUS DUAL GeForce RTX 3060 OC Edition Gaming G...,0,990.59,available
12,ASUS Dual GeForce RTX 3060 12GB GDDR6 PCI Expr...,0,1029.75,available
16,ASUS TUF Gaming GeForce RTX 3060 12GB GDDR6 PC...,0,1032.98,available
22,GIGABYTE GeForce RTX 3060 VISION OC 12G Graphi...,0,1040.98,available
30,MSI Ventus GeForce RTX 3060 12GB GDDR6 PCI Exp...,0,1045.90,available
